In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 在GCP上进行端到端机器学习：MLOps阶段6：使用Vertex AI在线预测开始使用XGBoost自定义模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_xgboost_model_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_xgboost_model_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage6/get_started_with_xgboost_model_online.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI工作台中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Vertex AI SDK部署XGBoost模型并进行在线预测。

### 目标

在本教程中，您将部署一个XGBoost模型，然后使用Vertex AI SDK进行在线预测。您也可以使用`gcloud`命令行工具或通过Cloud Console在线部署模型。

本教程使用以下谷歌云ML服务和资源：

- `Vertex AI Models`
- `Vertex AI Endpoints`
- `Vertex AI Prediction`
- `Vertex AI Pipelines`

执行的步骤包括：

- 将XGBoost模型上传为Vertex AI模型资源。
- 将模型部署到Vertex AI端点资源。
- 进行在线预测。
- 构建一个Vertex AI管道：
    - 将XGBoost模型上传为Vertex AI模型资源。
    - 将模型部署到Vertex AI端点资源。
- 进行在线预测。

### 模型

本教程使用的模型是一个预训练的XGBoost模型，该模型是在[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/iris)中的[Iris数据集](https://www.tensorflow.org/datasets/catalog/overview)上训练的。训练后的模型可以预测三种鸢尾花物种中的一种：山鸢尾(setosa)、维吉尼亚(virginica)或变色鸢尾(versicolor)。

### 价格

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成成本估算。

### 设置本地开发环境

如果您正在使用Colab或Vertex Workbench AI笔记本，您的环境已经满足运行该笔记本的所有要求。您可以跳过这一步。

否则，请确保您的环境满足该笔记本的要求。您需要以下内容：

- 云存储SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

云存储指南中的[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了一套简明的说明：

1. [安装并初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，在终端 shell 中运行 `pip3 install jupyter`。

5. 要启动Jupyter，在终端 shell 中运行 `jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

## 安装

安装以下软件包才能执行这个笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade --quiet {USER_FLAG} google-cloud-aiplatform \
                                             kfp \
                                             google-cloud-pipeline-components \
                                             tensorflow==2.5

重新启动内核

安装完额外的包后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

本教程不需要 GPU 运行时。

### 设置您的 Google Cloud 项目

**以下步骤是必需的，不论您的笔记本环境如何。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建一个帐户时，您会获得 $300 的免费信用额用于计算/存储成本。

2. [确保为您的项目启用了计费功能。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下 API：Vertex AI APIs、Compute Engine APIs 和 Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，您需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 为此笔记本中的所有命令使用正确的项目。

**注意**：Jupyter 运行以 `!` 为前缀的行作为 shell 命令，并插入以 `$` 为前缀的 Python 变量。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

地区

您还可以更改`REGION`变量，该变量用于本笔记本其余部分的操作。以下是Vertex AI支持的地区。我们建议您选择最靠近您的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能无法使用多区域存储桶来训练Vertex AI。并非所有地区都支持所有Vertex AI服务。

了解更多关于[Vertex AI地区](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

UUID

如果您在一个实时教程会话中，您可能正在使用一个共享的测试账户或项目。为了避免不同用户在创建资源时发生名称冲突，您可以为每个实例会话创建一个uuid，并将其附加到您在本教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的Google Cloud帐户

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经通过验证。

**如果您正在使用Colab**，运行下面的单元格，并按提示进行身份验证以通过oAuth验证您的帐户。

**否则**，请按以下步骤操作：

在Cloud控制台中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账户**。

在**服务账户名称**字段中输入一个名称，然后点击**创建**。

在**将此服务帐户授权给项目的权限**部分，点击权限下拉列表。在过滤框中键入"Vertex"，然后选择**Vertex管理员**。在过滤框中键入"Storage Object Admin"，然后选择**Storage Object Admin**。

点击创建。包含您的密钥的JSON文件将下载到本地环境中。

在下面的单元格中将您的服务帐户密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您的笔记本环境如何，以下步骤都是必需的。**

在初始化用于 Python 的 Vertex AI SDK 时，您需要指定一个云存储暂存桶。这个暂存桶是您的数据集和模型资源跨会话保留的地方。

在下方设置您的云存储桶的名称。桶的名称必须在所有 Google Cloud 项目中是全局唯一的，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = "gs://" + BUCKET_NAME

只有当您的存储桶还不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查云储存桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

服务帐户

**如果您不知道您的服务帐户**，请尝试使用`gcloud`命令通过执行下面的第二个单元格来获取您的服务帐户。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

print("Service Account:", SERVICE_ACCOUNT)

### 设置变量

接下来，设置一些在教程中使用的变量。

### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform

初始化顶点AI SDK，用于您的项目和相应的存储桶。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

#### 设置预构建的容器

Vertex为常用的ML框架提供了许多预构建的容器用于训练和预测。

设置用于预测的预构建的Docker容器镜像。

有关最新列表，请参见[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

有关最新列表，请参见[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
DEPLOY_VERSION = "xgboost-cpu.1-1"

LOCATION = REGION.split("-")[0]

DEPLOY_IMAGE = f"{LOCATION}-docker.pkg.dev/vertex-ai/prediction/{DEPLOY_VERSION}:latest"

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个虚拟CPU 3.75GB内存。
     - `n1-highmem`：每个虚拟CPU 6.5GB内存
     - `n1-highcpu`：每个虚拟CPU 0.9 GB内存
 - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96 \]

*注意：您还可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", DEPLOY_COMPUTE)

### 创建实例模式

instance.yaml文件用于验证输入请求的格式，在传递给模型服务器之前。如果发送的正确，您不需要instance.yaml。如果收到错误的请求，instance.yaml会提供比您从模型服务器获得的更有意义的错误消息。

*instance.yaml也可以作为模型监控自定义模型的输入模式重复使用。*

了解更多关于[Predict schemas](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/PredictSchemata)。

In [ ]:
yaml = """type: array
properties:
  sepal_length:
    type: numeric
  sepal_width:
    type: numeric
  petal_length:
    type: numeric
  petal_width:
    type: numeric
required:
  - sepal_length
  - sepal_width
  - petal_length
  - petal_width
"""

print(yaml)

with open("instance.yaml", "w") as f:
    f.write(yaml)

! gsutil cp instance.yaml {BUCKET_URI}/instance.yaml

### 将XGBoost模型的第一个版本上传到`Vertex AI模型`资源

接下来，您需要将模型的第一个版本作为`Model`资源上传到 `Vertex AI模型注册表`中，并设置以下参数：

- `display_name`：`Model`资源的可读名称
- `artifact_uri`：模型包的Cloud Storage位置
- `serving_container_image_uri`：用于提供服务的容器映像
- `is_default_version`：确定是否为`Model`资源的默认版本
- `version_ailiases`：模型版本的用户自定义替代别名列表，例如 `production`
- `version_description`：模型版本的用户描述

当第一个模型版本在`模型注册表`中创建时，属性`version_id`将自动设置为1。

了解有关 [使用Vertex AI模型注册表进行模型版本控制](https://cloud.google.com/vertex-ai/docs/model-registry/versioning) 的更多信息。

In [ ]:
MODEL_ARTIFACTS = (
    "gs://cloud-samples-data/vertex-ai/model-deployment/models/xgboost_iris"
)


model = aiplatform.Model.upload(
    display_name=f"xgboost_iris_{UUID}",
    serving_container_image_uri=DEPLOY_IMAGE,
    artifact_uri=MODEL_ARTIFACTS,
    instance_schema_uri=f"{BUCKET_URI}/instance.yaml",
    is_default_version=True,
    version_aliases=["v1", "version1"],
    version_description="This is the first version of the model",
    sync=True,
)

print(model.resource_name)

## 创建 `Endpoint` 资源

您可以使用 `Endpoint.create()` 方法来创建 `Endpoint` 资源。至少，您需要指定端点的显示名称。可选地，您可以指定项目和位置（区域）；否则，设置将继承您使用 `init()` 方法初始化 Vertex AI SDK 时设置的值。

在此示例中，指定了以下参数：

- `display_name`：`Endpoint` 资源的可读名称。
- `project`：您的项目 ID。
- `location`：您的区域。
- `labels`：（可选）以键/值对形式定义的用于 `Endpoint` 的用户定义元数据。

此方法将返回一个 `Endpoint` 对象。

了解更多关于[Vertex AI Endpoints](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)。

In [ ]:
endpoint = aiplatform.Endpoint.create(
    display_name="xgboost_iris_" + UUID,
    project=PROJECT_ID,
    location=REGION,
    labels={"your_key": "your_value"},
)

print(endpoint)

部署`Model`资源到`Endpoint`资源

您可以部署一个或多个`Vertex AI Model`资源实例到同一个终端点。每个部署的`Vertex AI Model`资源将有自己的用于服务二进制的部署容器。

*注意:* 对于这个示例，您的XGBoost模型的部署容器是预定义的。

### 扩展

`Vertex AI Endpoint`资源支持三种类型的扩展：

- 无扩展：服务二进制部署到单个VM实例。
- 手动扩展：服务二进制部署到固定数量的多个VM实例。
- 自动扩展：根据负载情况，服务二进制部署到的VM实例数量会有所变化。

### 手动扩展

在下一个示例中，您可以将 `Vertex AI Model` 资源部署到 `Vertex AI Endpoint` 资源以进行手动扩展 -- 固定数量（大于1）的 VM 实例。换句话说，当模型被部署时，固定数量的 VM 实例将被规划，并保持规划状态直到模型被撤销。

在此示例中，您可以按照以下最小指定参数部署模型:

- `model`: 要部署的 `Model` 资源。
- `machine_type`: 每个 VM 实例的机器类型。
- `deployed_model_displayed_name`: 部署模型实例的人类可读名称。
- `min_replica_count`: 要规划的最小 VM 实例（节点）数量。
- `max_replica_count`: 要规划的最大 VM 实例（节点）数量。
- `accelerator_type`: 每个 VM 实例的硬件加速器类型，如果有的话。
- `accelerator_count`: 每个 VM 实例的硬件加速器数量，如果有的话。

对于手动扩展，固定数量的 VM 实例将在部署模型期间被规划。

*注意:* 对于手动扩展，节点的最小和最大数量被设置为相同的值。

In [ ]:
MIN_NODES = 2
MAX_NODES = 2


response = endpoint.deploy(
    model=model,
    deployed_model_display_name=f"xgboost_iris_{UUID}",
    machine_type=DEPLOY_COMPUTE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
)

发送一个在线预测请求

向您部署的模型发送一个在线预测。

制作一个测试项目

您可以从数据集中选择一个任意的例子作为测试项目。

In [ ]:
TEST_INSTANCE = [5.1, 3.5, 1.4, 0.2]

### 进行预测

现在您的`Model`资源已部署到`Endpoint`资源，您可以通过向Endpoint资源发送预测请求来进行在线预测。

#### 请求

对于XGBoost模型，模型服务器将输入视为一组数字值，其中数值的顺序与XGBoost模型训练时所用特征的顺序相同。

    [ numeric_value_1, numeric_value_2, .... ]

#### 响应

从`predict()`调用的响应是一个Python字典，具有以下条目：

- `prediction`：预测的类别。
- `deployed_model_id`：进行预测的Vertex AI部署的Model资源的标识符。
- `model_version_id`：模型版本。

*注意：*预测的类别将作为浮点值返回。因此，0.0代表标签0。

In [ ]:
prediction = endpoint.predict(instances=[TEST_INSTANCE])

print(prediction)

### 下线模型

当您完成预测后，您需要从`端点`资源中下线模型。这将取消所有计算资源并停止部署模型的计费。

In [ ]:
endpoint.undeploy_all()
endpoint.delete()

删除模型

接下来，您可以使用`delete（）`方法删除`Vertex AI模型`资源。

In [ ]:
model.delete()

## 创建一个 Vertex AI 管道

接下来，您会重复与 Vertex AI 管道相同的步骤，如下：

1. 创建一个组件，将预训练模型上传并版本化到 Vertex AI Model Registry。
2. 创建一个 Vertex AI 管道定义。
3. 编译管道。
4. 执行管道。

### 创建带有版本组件的模型上传

首先，您可以创建自己的组件来上传一个预先训练好的模型到 Vertex AI 模型注册表，并设置模型版本参数。您的组件的输出产物是一个 VertexModel。

In [ ]:
import kfp
from kfp.v2.dsl import component


@component(
    base_image="python:3.9",
    packages_to_install=["google-cloud-aiplatform", "google-cloud-pipeline-components"],
)
def model_upload_version(
    project: str,
    display_name: str,
    model_artifacts: str,
    is_default_version: bool,
    version_aliases: list,
    version_description: str,
    serving_container_image_uri: str,
) -> str:
    import google.cloud.aiplatform as aiplatform

    aiplatform.init(project=project)

    model = aiplatform.Model.upload(
        display_name=display_name,
        serving_container_image_uri=serving_container_image_uri,
        artifact_uri=model_artifacts,
        is_default_version=is_default_version,
        version_aliases=version_aliases,
        version_description=version_description,
        sync=True,
    )
    return model.resource_name

创建管道

接下来，您可以通过以下步骤创建一个管道：

- 上传并版本化预训练模型的工件。
- 创建一个 Endpoint 资源。
- 将模型资源部署到 Endpoint 资源中。

In [ ]:
@kfp.dsl.pipeline(name=f"xgboost-iris-{UUID}")
def pipeline(
    display_name: str,
    is_default_version: bool,
    version_aliases: list,
    version_description: str,
    model_artifacts: str,
    serving_container_image_uri: str,
    machine_type: str,
    project: str = PROJECT_ID,
):
    from google_cloud_pipeline_components.experimental.evaluation import \
        GetVertexModelOp
    from google_cloud_pipeline_components.v1.endpoint import (EndpointCreateOp,
                                                              ModelDeployOp)

    model_upload_op = model_upload_version(
        project=project,
        display_name=display_name,
        is_default_version=is_default_version,
        version_aliases=version_aliases,
        version_description=version_description,
        model_artifacts=model_artifacts,
        serving_container_image_uri=serving_container_image_uri,
    )

    model = GetVertexModelOp(model_resource_name=model_upload_op.output)

    endpoint_create_op = EndpointCreateOp(project=project, display_name=display_name)

    _ = ModelDeployOp(
        endpoint=endpoint_create_op.outputs["endpoint"],
        # model=model_upload_op.outputs["model"],
        model=model.outputs["model"],
        deployed_model_display_name=display_name,
        dedicated_resources_machine_type=DEPLOY_COMPUTE,
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=1,
    )

编译流水线。

In [ ]:
from kfp.v2 import compiler  # noqa: F811

compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path=f"xgboost-iris-{UUID}.json",
)

### 执行管道

接下来，您执行管道。管道接受以下参数，这些参数作为字典`parameter_values`传递：

- `project`：您的项目ID
- `display_name`：`Model`资源的人类可读名称。
- `model_artifacts`：模型包的Cloud Storage位置。
- `serving_container_image_uri`：服务容器镜像。
- `is_default_version`：这将是`Model`资源的默认版本。
- `version_ailiases`：模型版本的用户定义的替代别名列表，比如`production`。
- `version_description`：模型版本的用户描述。
- `machine_type`：部署VM的机器类型。

In [ ]:
PIPELINE_ROOT = f"{BUCKET_URI}/pipeline_root/xgboost_iris"

pipeline = aiplatform.PipelineJob(
    display_name=f"xgboost_iris_{UUID}",
    template_path=f"xgboost-iris-{UUID}.json",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={
        "project": PROJECT_ID,
        "display_name": f"xgboost_iris_{UUID}",
        "is_default_version": True,
        "version_aliases": ["v1", "version1"],
        "version_description": "This is the first version",
        "model_artifacts": MODEL_ARTIFACTS,
        "serving_container_image_uri": DEPLOY_IMAGE,
        "machine_type": DEPLOY_COMPUTE,
    },
    enable_caching=False,
)

pipeline.run()

! rm -f xgboost-iris-{UUID}.json

查看管道结果

最后，您将查看管道中每个任务的工件输出。

In [ ]:
import json

import tensorflow as tf

PROJECT_NUMBER = pipeline.gca_resource.name.split("/")[1]
print(PROJECT_NUMBER)


def print_pipeline_output(job, output_task_name):
    JOB_ID = job.name
    print(JOB_ID)
    for _ in range(len(job.gca_resource.job_detail.task_details)):
        TASK_ID = job.gca_resource.job_detail.task_details[_].task_id
        EXECUTE_OUTPUT = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/executor_output.json"
        )
        GCP_RESOURCES = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/gcp_resources"
        )
        EVAL_METRICS = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/evaluation_metrics"
        )
        if tf.io.gfile.exists(EXECUTE_OUTPUT):
            ! gsutil cat $EXECUTE_OUTPUT
            return EXECUTE_OUTPUT
        elif tf.io.gfile.exists(GCP_RESOURCES):
            ! gsutil cat $GCP_RESOURCES
            return GCP_RESOURCES
        elif tf.io.gfile.exists(EVAL_METRICS):
            ! gsutil cat $EVAL_METRICS
            return EVAL_METRICS

    return None


print("get-vertex-model")
artifacts = print_pipeline_output(pipeline, "get-vertex-model")
print("\n\n")
output = !gsutil cat $artifacts
output = json.loads(output[0])
model_id = output["artifacts"]["model"]["artifacts"][0]["metadata"]["resourceName"]
print("\n\n")
print("endpoint-create")
artifacts = print_pipeline_output(pipeline, "endpoint-create")
print("\n\n")
output = !gsutil cat $artifacts
output = json.loads(output[0])
endpoint_id = output["artifacts"]["endpoint"]["artifacts"][0]["metadata"][
    "resourceName"
]
print("model-deploy")
artifacts = print_pipeline_output(pipeline, "model-deploy")
print("\n\n")

### 进行预测

最后，您对通过流水线部署到终端点的XGBoost模型进行预测。

In [ ]:
endpoint = aiplatform.Endpoint(endpoint_id)

prediction = endpoint.predict(instances=[TEST_INSTANCE])
print(prediction)

删除管道作业

在管道作业完成后，您可以使用`delete（）`方法删除管道作业。在完成之前，可以使用`cancel（）`方法取消管道作业。

In [ ]:
pipeline.delete()

删除顶点模型和端点。

接下来，部署并删除顶点模型和端点资源。

In [ ]:
endpoint.undeploy_all()
endpoint.delete()

model = aiplatform.Model(model_id)
model.delete()

清理工作

要清理本项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
delete_bucket = True

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}